<div class="alert alert-block alert-info"> 
- __[Shortcuts](https://www.cheatography.com/weidadeyue/cheat-sheets/jupyter-notebook/)__
- __[Markdown](http://datascience.ibm.com/blog/markdown-for-jupyter-notebooks-cheatsheet/)__

# Recolha dados BD

In [6]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### Setup

In [7]:
pd.set_option('display.max_columns', 500)

### Criação do objecto de ligação à MySQL DB utilizando o módulo PyMySQL

In [75]:
%%capture 
with open('password.txt') as f: mysqlpass = f.read()

In [76]:
engine = create_engine('mysql+pymysql://grupo_1:{}@151.236.42.86:3306/iefp'.format(mysqlpass))
connection = engine.raw_connection()

### Carregamento das tabelas em variáveis (carregar apenas as linhas necessárias) 

In [36]:
pedido_fim_2016 = pd.read_sql('SELECT * FROM pedido_fim_2016', connection)

In [51]:
pedidos_inscritos_longos = pd.read_sql('SELECT * FROM pedidos_inscritos_longos', connection)

In [30]:
sie_31 = pd.read_sql('SELECT * FROM sie_31', connection)

In [31]:
sie_35 = pd.read_sql('SELECT * FROM sie_35', connection)

In [32]:
sie_36 = pd.read_sql('SELECT * FROM sie_36', connection)

In [33]:
sie_37 = pd.read_sql('SELECT * FROM sie_37', connection)

In [34]:
sie_38 = pd.read_sql('SELECT * FROM sie_38', connection)

In [35]:
sie_43 = pd.read_sql('SELECT * FROM sie_43', connection)

### **Observações:**

In [69]:
print("Número de registos das tabelas:")
print('{:>25}  {:>6}'.format("pedido_fim_2016:", len(pedido_fim_2016)))
print('{:>25}  {:>6}'.format("pedidos_inscritos_longos:", len(pedidos_inscritos_longos)))
print('{:>25}  {:>6}'.format("sie_31:", len(sie_31)))
print('{:>25}  {:>6}'.format("sie_35:", len(sie_35)))
print('{:>25}  {:>6}'.format("sie_36:", len(sie_36)))
print('{:>25}  {:>6}'.format("sie_37:", len(sie_37)))
print('{:>25}  {:>6}'.format("sie_38:", len(sie_38)))
print('{:>25}  {:>6}'.format("sie_43:", len(sie_43)))

Número de registos das tabelas:
         pedido_fim_2016:    7995
pedidos_inscritos_longos:  125029
                  sie_31:  114261
                  sie_35:  458216
                  sie_36:  133309
                  sie_37:  155644
                  sie_38:  265994
                  sie_43:  125554


**pedido_fim_2016:**<br>
Não tem UteId's Repetidos (ao contrário de pedidos_inscritos_longos)<br>
5974 registos UteId unicos <br>
'SELECT COUNT(DISTINCT(b.UteId)) FROM pedido_fim_2016 a RIGHT JOIN pedidos_inscritos_longos b ON a.UteId=b.UteId WHERE a.UteId IS NULL'<br>
<br>
**pedidos_inscritos_longos:**<br> 
63502 registos UteId unicos em pedidos_inscritos_longos: <br>
'SELECT COUNT(DISTINCT(b.UteId)) FROM pedido_fim_2016 a RIGHT JOIN pedidos_inscritos_longos b ON a.UteId=b.UteId WHERE a.UteId IS NULL'          

<img src=".\images\UteId_Intersections_2016_longos.jpg" title="UteId intersections between pedidos_inscritos_longos e pedido_fim_2016" />

### Fechar a ligação

In [82]:
connection.close()